<a href="https://colab.research.google.com/github/kaushal27120/My_Projects/blob/main/Spam_mail_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
!pip install -q transformers datasets


In [20]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from transformers import DataCollatorWithPadding

fake = pd.read_csv('/content/Fake.csv')
real = pd.read_csv('/content/True.csv')

fake['label'] = 0
real['label'] = 1

df = pd.concat([fake, real]).sample(frac=1).reset_index(drop=True)
df['text'] = df['title'] + ' ' + df['text']

df = df.sample(n=2000, random_state=42).reset_index(drop=True)


In [21]:
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['label'], test_size=0.2, random_state=42)


In [22]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

train_encodings = tokenizer(list(X_train), truncation=True, padding=True, max_length=512)
test_encodings = tokenizer(list(X_test), truncation=True, padding=True, max_length=512)


In [23]:
class FakeNewsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, idx):
        return {k: torch.tensor(v[idx]) for k, v in self.encodings.items()} | {'labels': torch.tensor(self.labels[idx])}
    def __len__(self):
        return len(self.labels)

train_dataset = FakeNewsDataset(train_encodings, list(y_train))
test_dataset = FakeNewsDataset(test_encodings, list(y_test))


In [24]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [25]:
training_args = TrainingArguments(
    output_dir='./results',
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1,
    logging_dir='./logs',
    logging_steps=10,
    save_total_limit=1,
    do_train=True,
    do_eval=True
)

In [26]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=DataCollatorWithPadding(tokenizer)
)

In [27]:
trainer.train()

Step,Training Loss
10,0.615200
20,0.298000
30,0.138800
40,0.092500
50,0.108900
60,0.004800
70,0.122700
80,0.093000
90,0.161300
100,0.459500


TrainOutput(global_step=200, training_loss=0.10790551385143772, metrics={'train_runtime': 162.4477, 'train_samples_per_second': 9.849, 'train_steps_per_second': 1.231, 'total_flos': 420977688576000.0, 'train_loss': 0.10790551385143772, 'epoch': 1.0})

In [28]:
predictions = trainer.predict(test_dataset)
pred_labels = predictions.predictions.argmax(-1)

print(classification_report(y_test, pred_labels))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00       211
           1       1.00      1.00      1.00       189

    accuracy                           1.00       400
   macro avg       1.00      1.00      1.00       400
weighted avg       1.00      1.00      1.00       400



In [29]:
from sklearn.metrics import accuracy_score

acc = accuracy_score(y_test, pred_labels)
print("Accuracy:", round(acc * 100, 2), "%")

Accuracy: 100.0 %


In [30]:
print("\nFirst 10 Predictions vs Actual Labels:")
for i in range(10):
    print(f"{i+1}. Predicted: {pred_labels[i]}, Actual: {list(y_test)[i]}")


First 10 Predictions vs Actual Labels:
1. Predicted: 1, Actual: 1
2. Predicted: 0, Actual: 0
3. Predicted: 1, Actual: 1
4. Predicted: 1, Actual: 1
5. Predicted: 1, Actual: 1
6. Predicted: 0, Actual: 0
7. Predicted: 1, Actual: 1
8. Predicted: 1, Actual: 1
9. Predicted: 1, Actual: 1
10. Predicted: 0, Actual: 0
